In [1]:
import os
import pandas as pd
import numpy as np
from combat.pycombat import pycombat
from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import joblib

In [2]:
os.chdir("../../Dataset/Merged")

In [4]:
dataset = pd.read_csv('MergedDataset-136411.csv', index_col=0)

In [5]:
sampleID = dataset['SampleID']
datasetID = dataset['SampleID'].apply(lambda x: x.split('-')[0]).values
indicator = dataset['Label']
dataset = dataset.drop(columns=['SampleID', 'Label'])

dataset = pycombat(dataset.transpose(), datasetID).transpose()

dataset.insert(0, 'SampleID', sampleID)
dataset.insert(1, 'Label', indicator)

Found 6 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.


C:\Users\samue\AppData\Roaming\Python\Python311\site-packages\combat\pycombat.py:159: RuntimeWarning: divide by zero encountered in divide
  np.absolute(d_new-d_old)/d_old))  # maximum difference between new and old estimate


Adjusting the Data


In [6]:
def getPatientID(sampleID):
    return sampleID.split('-')[0] + '-' + sampleID.split('-')[1].split('_', 1)[1]

dataset.insert(1, 'PatientID', dataset['SampleID'].apply(getPatientID))

print(dataset)

gruppi = dataset.groupby('PatientID')

def sanity_check(gruppi):
    for group_name, group_data in gruppi:
        if 'Control' in group_data['SampleID'].iloc[0]:
            for e in group_data['SampleID']:
                if not 'Control' in e:
                    print("Errore in gruppo:", group_name)
                    break
        else:
            for e in group_data['SampleID']:
                if 'Control' in e:
                    print("Errore in gruppo:", group_name)
                    break

sanity_check(gruppi)

                                         SampleID      PatientID  Label  \
0                        0-GSM1026056_600009.0001  0-600009.0001      1   
1             0-GSM1026057_600009.0001-FollowUp_1  0-600009.0001      1   
2                         0-GSM1026058_41461.0001   0-41461.0001      1   
3                         0-GSM1026059_41462.0001   0-41462.0001      1   
4                        0-GSM1026060_600029.0001  0-600029.0001      1   
...                                           ...            ...    ...   
1799  5-GSM2347715_NT142_W18D2-Control-FollowUp_1  5-NT142_W18D2      0   
1800  5-GSM2347717_NT041_W18D2-Control-FollowUp_2  5-NT041_W18D2      0   
1801  5-GSM2347719_NT142_W18D2-Control-FollowUp_2  5-NT142_W18D2      0   
1802  5-GSM2347721_NT041_W18D2-Control-FollowUp_3  5-NT041_W18D2      0   
1803  5-GSM2347723_NT142_W18D2-Control-FollowUp_3  5-NT142_W18D2      0   

        STEEP1   SEC14L1     YIPF5    SLC1A5        C2      NOL6     SPRR3  \
0     5.140886  8.939

In [7]:
#n_splits number of re-shuffling & splitting iterations.
#test_size represents the proportion of the dataset to include in the test split.
#random_state is the seed used by the random number generator.
splitter = GroupShuffleSplit(n_splits=2, test_size=0.25, random_state = 42)
split = splitter.split(dataset, groups=dataset['PatientID'])
train_inds, test_inds = next(split)

train = dataset.iloc[train_inds].sample(frac=1, random_state=42)
test = dataset.iloc[test_inds].sample(frac=1, random_state=42)

print("Dataset di train:")
print(train.shape)
print("I malati sono: ", sum(train['Label'] == 1))
print("I sani sono: ", sum(train['Label'] == 0))

print("\nDataset di test:")
print(test.shape)
print("I malati sono: ", sum(test['Label'] == 1))
print("I sani sono: ", sum(test['Label'] == 0))

#malati train 45.12 perc
#malati test 42.49 perc
#malati sul totale 44.49 perc

y_train = train['Label']
x_train = train.drop(columns=['SampleID', 'Label', 'PatientID'])

y_test = test['Label']
x_test = test.drop(columns=['SampleID', 'Label', 'PatientID'])

Dataset di train:
(1340, 12094)
I malati sono:  521
I sani sono:  819

Dataset di test:
(464, 12094)
I malati sono:  177
I sani sono:  287


In [8]:
def trainModel(model, hyperparameters, x_train, y_train, printTrain, njobs): 
    pipeline = Pipeline(steps=[('Scaling', MinMaxScaler()), ('classifier', model)])
    gridSearch = GridSearchCV(pipeline, param_grid=hyperparameters, cv=5, return_train_score=True, refit=True, n_jobs=njobs, verbose=10, error_score='raise') if printTrain == True else GridSearchCV(pipeline, param_grid=hyperparameters, cv=5, return_train_score=False, refit=True, n_jobs=njobs, verbose=10, error_score='raise')
    gridSearch.fit(x_train, y_train)
    print("Best model:", gridSearch.best_estimator_, gridSearch.best_params_)
    print("Best score:", np.max(gridSearch.cv_results_['mean_test_score']))
    print("All scores:", gridSearch.cv_results_['mean_test_score'])
    return gridSearch

In [ ]:
svc = trainModel(SVC(), {
    'classifier__kernel': ['linear', 'poly', 'rbf'], 
    'classifier__probability': [True],
    'classifier__C': [2**-5, 2**-3, 2**-1, 2**0, 2**1, 2**3, 2**7, 2**9, 2**11, 2**13, 2**15],
    'classifier__gamma': [2**-15, 2**-13, 2**-11, 2**-9, 2**-7, 2**-5, 2**-3, 2**-1, 2**1, 2**3, 2**5, 'scale', 'auto']}, x_train, y_train, False, -1)

In [9]:
svc = joblib.load('../../Modelli/Dataset-136411/svc.pkl')
print("Best model:", svc.best_estimator_, svc.best_params_)
print("Best score:", np.max(svc.cv_results_['mean_test_score']))
print("All scores:", svc.cv_results_['mean_test_score'])

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier', SVC(C=128, gamma=0.0078125, probability=True))]) {'classifier__C': 128, 'classifier__gamma': 0.0078125, 'classifier__kernel': 'rbf', 'classifier__probability': True}
Best score: 0.9783582089552239
All scores: [0.59328358 0.61119403 0.61119403 0.59328358 0.61119403 0.61119403
 0.59328358 0.61119403 0.61119403 0.59328358 0.70671642 0.61119403
 0.59328358 0.88059701 0.61119403 0.59328358 0.88059701 0.61119403
 0.59328358 0.88059701 0.61119403 0.59328358 0.88059701 0.61119403
 0.59328358 0.88059701 0.61119403 0.59328358 0.88059701 0.61119403
 0.59328358 0.88059701 0.61119403 0.59328358 0.87985075 0.61119403
 0.59328358 0.61119403 0.61119403 0.60671642 0.61119403 0.61119403
 0.60671642 0.61119403 0.61119403 0.60671642 0.61119403 0.61119403
 0.60671642 0.87238806 0.61119403 0.60671642 0.88059701 0.64776119
 0.60671642 0.88059701 0.61119403 0.60671642 0.88059701 0.61119403
 0.60671642 0.88059701 0.61119403

In [ ]:
randomForest = trainModel(RandomForestClassifier(), {
    'classifier__n_estimators': [i for i in range(150, 251, 25)],
    'classifier__max_depth': [i for i in range(4, 13)]}, x_train, y_train, True, -1)

In [10]:
randomForest = joblib.load('../../Modelli/Dataset-136411/randomForest.pkl')
print("Best model:", randomForest.best_estimator_, randomForest.best_params_)
print("Best score:", np.max(randomForest.cv_results_['mean_test_score']))
print("All scores:", randomForest.cv_results_['mean_test_score'])

y_pred = randomForest.predict(x_test)
report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(report)

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=12, n_estimators=225))]) {'classifier__max_depth': 12, 'classifier__n_estimators': 225}
Best score: 0.9723880597014926
All scores: [0.92686567 0.9261194  0.92761194 0.92835821 0.92761194 0.94552239
 0.94328358 0.94104478 0.94328358 0.94253731 0.95746269 0.95447761
 0.95522388 0.95746269 0.95597015 0.95895522 0.96044776 0.96567164
 0.96268657 0.9641791  0.96567164 0.96716418 0.97089552 0.96567164
 0.96791045 0.96567164 0.9619403  0.96641791 0.96567164 0.96343284
 0.96641791 0.96791045 0.96716418 0.96791045 0.96567164 0.96865672
 0.96716418 0.96791045 0.96492537 0.97089552 0.96940299 0.96791045
 0.96865672 0.97238806 0.96641791]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       287
           1       0.90      0.94      0.92       177

    accuracy                           

In [ ]:
elasticNet = trainModel(LogisticRegression(), {
    'classifier__C': [2**-7, 2**-5, 2**-3, 2**-1, 2**0, 2**1, 2**3, 2**7, 2**9, 2**11],
    'classifier__penalty': ['elasticnet'],
    'classifier__l1_ratio': [0, 0.0001, 0.1, 0.25, 0.5, 0.75, 1],
    'classifier__solver': ['saga']}, x_train, y_train, True, -1)

In [11]:
elasticNet = joblib.load('../../Modelli/Dataset-136411/elasticNet.pkl')
print("Best model:", elasticNet.best_estimator_, elasticNet.best_params_)
print("Best score:", np.max(elasticNet.cv_results_['mean_test_score']))
print("All scores:", elasticNet.cv_results_['mean_test_score'])

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier',
                 LogisticRegression(C=0.5, l1_ratio=1, penalty='elasticnet',
                                    solver='saga'))]) {'classifier__C': 0.5, 'classifier__l1_ratio': 1, 'classifier__penalty': 'elasticnet', 'classifier__solver': 'saga'}
Best score: 0.7283582089552239
All scores: [0.62835821 0.62910448 0.61119403 0.61119403 0.61119403 0.61119403
 0.61119403 0.62089552 0.62164179 0.64029851 0.6119403  0.61119403
 0.61119403 0.61119403 0.60447761 0.60447761 0.66791045 0.70671642
 0.66567164 0.63656716 0.63059701 0.60298507 0.60447761 0.62537313
 0.65597015 0.69179104 0.71940299 0.72835821 0.60074627 0.60298507
 0.61567164 0.62761194 0.65671642 0.67835821 0.69402985 0.60373134
 0.60223881 0.60820896 0.61716418 0.62910448 0.64179104 0.66044776
 0.60447761 0.60223881 0.60223881 0.60746269 0.61119403 0.61567164
 0.61791045 0.60149254 0.60149254 0.60223881 0.60298507 0.60223881
 0.60373134 0.6029

In [ ]:
knn = trainModel(KNeighborsClassifier(), {
    'classifier__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'classifier__weights': ['uniform', 'distance'] }, x_train, y_train, True, 3)

In [12]:
knn = joblib.load('../../Modelli/Dataset-136411/knn.pkl')
print("Best model:", knn.best_estimator_, knn.best_params_)
print("Best score:", np.max(knn.cv_results_['mean_test_score']))
print("All scores:", knn.cv_results_['mean_test_score'])

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier', KNeighborsClassifier(n_neighbors=3))]) {'classifier__n_neighbors': 3, 'classifier__weights': 'uniform'}
Best score: 0.7970149253731342
All scores: [0.79701493 0.79701493 0.78208955 0.78208955 0.7761194  0.7761194
 0.76865672 0.76865672 0.76791045 0.76791045 0.76716418 0.76716418
 0.76119403 0.76119403]


In [ ]:
HistGradientBoosting = trainModel(HistGradientBoostingClassifier(), {
    'classifier__max_depth': [3, 5, 7, 9, 11],
    'classifier__max_iter': [100, 150, 200, 250],
    'classifier__learning_rate': [0.1, 0.01, 0.001]}, x_train, y_train, False, 4)

In [13]:
HistGradientBoosting = joblib.load('../../Modelli/Dataset-136411/HistGradientBoosting.pkl')
print("Best model:", HistGradientBoosting.best_estimator_, HistGradientBoosting.best_params_)
print("Best score:", np.max(HistGradientBoosting.cv_results_['mean_test_score']))
print("All scores:", HistGradientBoosting.cv_results_['mean_test_score'])

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier', HistGradientBoostingClassifier(max_depth=9))]) {'classifier__learning_rate': 0.1, 'classifier__max_depth': 9, 'classifier__max_iter': 100}
Best score: 0.9783582089552239
All scores: [0.97164179 0.97537313 0.97686567 0.97761194 0.97686567 0.97686567
 0.9761194  0.97537313 0.9738806  0.9738806  0.97537313 0.97537313
 0.97835821 0.97537313 0.97537313 0.9761194  0.97686567 0.9761194
 0.9761194  0.97686567 0.95373134 0.96567164 0.96865672 0.97164179
 0.95522388 0.96641791 0.97089552 0.97537313 0.95671642 0.96343284
 0.97164179 0.9761194  0.95671642 0.96268657 0.97238806 0.97537313
 0.95671642 0.96268657 0.97014925 0.97462687 0.61119403 0.61119403
 0.64925373 0.85373134 0.61119403 0.61119403 0.64328358 0.89701493
 0.61119403 0.61119403 0.64701493 0.89925373 0.61119403 0.61119403
 0.64850746 0.89925373 0.61119403 0.61119403 0.64850746 0.89925373]


In [ ]:
GradientBoosting = trainModel(GradientBoostingClassifier(), {
    'classifier__max_depth': [3, 5, 7, 9, 11],
    'classifier__max_iter': [100, 150, 200, 250],
    'classifier__learning_rate': [0.1, 0.01, 0.001]}, x_train, y_train, False, 4)

In [15]:
GradientBoosting = joblib.load('../../Modelli/Dataset-136411/GradientBoosting.pkl')
print("Best model:", GradientBoosting.best_estimator_, GradientBoosting.best_params_)
print("Best score:", np.max(GradientBoosting.cv_results_['mean_test_score']))
print("All scores:", GradientBoosting.cv_results_['mean_test_score'])

Best model: Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('classifier', GradientBoostingClassifier(n_estimators=200))]) {'classifier__learning_rate': 0.1, 'classifier__loss': 'log_loss', 'classifier__max_depth': 3, 'classifier__n_estimators': 200}
Best score: 0.9753731343283581
All scores: [0.97164179 0.97462687 0.97537313 0.96343284 0.96268657 0.96044776
 0.92238806 0.92537313 0.92761194 0.94104478 0.95671642 0.96119403
 0.94029851 0.94328358 0.94402985 0.92313433 0.92238806 0.92313433
 0.61119403 0.61119403 0.6119403  0.61119403 0.61119403 0.61268657
 0.61119403 0.61119403 0.61119403]


In [ ]:
naiveBayes = Pipeline(steps=[('Scaling', MinMaxScaler()), ('classifier', GaussianNB())])
print(naiveBayes.fit(x_train, y_train).score(x_train, y_train))

In [16]:
naiveBayes = joblib.load('../../Modelli/Dataset-136411/naiveBayes.pkl')
print(naiveBayes.score(x_train, y_train))

0.9238805970149254


Performance dei vari modelli

In [17]:
def prettyPrint(model, name):
    if isinstance(model, Pipeline):
        print(name + ":")
        print("Iperparametri: ", model)
        print("Training accuracy: ", model.score(x_train, y_train))
        print("Test accuracy: ", model.score(x_test, y_test), "\n")
        print(classification_report(y_test, model.predict(x_test)), "\n\n")
    else:
        print(name + ":")
        print("Iperparametri: ", model.best_params_)
        print("Training accuracy: ", model.best_score_)
        print("Test accuracy: ", model.score(x_test, y_test), "\n")
        print(classification_report(y_test, model.predict(x_test)), "\n\n")

for model, name in [(svc, "SVC"), (randomForest, "Random Forest"), (elasticNet, "Elastic Net"), (knn, "KNN"), (HistGradientBoosting, "Hist Gradient Boosting"), (GradientBoosting, "Gradient Boosting"), (naiveBayes, "Naive Bayes")]:
    prettyPrint(model, name)

SVC:
Iperparametri:  {'classifier__C': 128, 'classifier__gamma': 0.0078125, 'classifier__kernel': 'rbf', 'classifier__probability': True}
Training accuracy:  0.9783582089552239
Test accuracy:  0.9525862068965517 

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       287
           1       0.92      0.96      0.94       177

    accuracy                           0.95       464
   macro avg       0.95      0.95      0.95       464
weighted avg       0.95      0.95      0.95       464
 


Random Forest:
Iperparametri:  {'classifier__max_depth': 12, 'classifier__n_estimators': 225}
Training accuracy:  0.9723880597014926
Test accuracy:  0.9353448275862069 

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       287
           1       0.90      0.94      0.92       177

    accuracy                           0.94       464
   macro avg       0.93      0.94      0.93       464
weighted avg  